# Betamaps exploration

This script creates handy visualisations of betamaps calculated on previous step. For each subject and task condition single plot is created. Each plot consist six subplots:
- **top panel**: framewise displacement (FD) for entire run duration, it roughly corresponds (visually) to trials from middle panel
- **middle panel**: beta maps carpetplot (voxels x trials); beta maps are masked before effect size extraction
- **bottom-left panel**: mean beta map for +PE and -PE trials displayed in brain space
- **bottom-right panel**: histogram from carpetplot values marginalized over voxels and trials 

Figures are stored as `.png` files named according to BIDS-like convention:

> `betamaps_figures/sub-<subject_label>_task-<taks_label>_betamap_carpetplot.png`

Note that beta maps are independent of atlas choice. Therefore beta maps GLM can be done only once and used later for different parcellations. 

In [ ]:
import json
from os.path import join
from pathlib import Path

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import pandas as pd
from dn_utils.behavioral_models import load_behavioral_data
from dn_utils.path import path
from dn_utils.plotting import aligned_imshow_cbar
from mpl_toolkits.axes_grid1 import make_axes_locatable

%matplotlib inline

In [ ]:
# Settings
clim_mean_betamaps = [-25, 25]
clim_carpet = [-500, 500]
hist_range = (-25, 25)
z_slice = 25

In [ ]:
def array_coords_to_mni(img, i, j, k):
    """Calculate X, Y, Z coordinates for i, j, k."""
    M = img.affine[:3, :3]
    abc = img.affine[:3, 3]
    return M.dot([i, j, k]) + abc

In [ ]:
# Create paths
path_betamaps = join(path["bsc"], "betamaps")
path_betamaps_figures = join(path["bsc"], "betamaps_figures")
Path(path_betamaps_figures).mkdir(exist_ok=True)

# Load behavioral data
beh, meta = load_behavioral_data(path["behavioral"], verbose=False)

# Load masks
with open(join(path["data_paths"], "mask_filenames.json"), "r") as f:
    mask_files = json.loads(f.read())   
with open(join(path["data_paths"], "conf_filenames.json"), "r") as f:
    conf_files = json.loads(f.read())   
    
# Load betamats
imgs, masks = {}, {}
for con_idx, con in enumerate(meta["dim2"]):
    con_name = f"prl{con}"
    imgs[con_name], masks[con_name] = [], []
    for sub_idx, sub in enumerate(meta["dim1"]):
        img_fname = f"sub-{sub}_task-{con_name}_betamaps.nii.gz" 
        imgs[con_name].append(nib.load(join(path_betamaps, img_fname)))
        masks[con_name].append(nib.load(mask_files[con_name][sub_idx]))

In [ ]:
for con_idx, con in enumerate(meta["dim2"]):
    for sub_idx, sub in enumerate(meta["dim1"]):
        con_name = f"prl{con}"
        
        # Load prediciton error sign for all trials
        won_bool = beh[sub_idx, con_idx, :, meta["dim4"].index("won_bool")]
        won_bool = won_bool.astype(bool)
        n_pos_trials = sum(won_bool)

        # Load betamats & brain mask & confounds
        img = imgs[con_name][sub_idx]
        mask = masks[con_name][sub_idx]
        img_data = img.get_fdata()
        mask_data = mask.get_fdata()
        df_conf = pd.read_csv(conf_files[con_name][sub_idx], delimiter="\t")
        df_conf["framewise_displacement"]
        
        # Calulate brain coord
        if con_idx == 0 and sub_idx == 0:
            _, _, z_mni = array_coords_to_mni(img, 0, 0, z_slice)
        
        # Calculate carpetplots
        carpet = img_data[mask_data.astype("bool")]
        b_trials = np.mean(carpet, axis=0)
        b_voxels = np.mean(carpet, axis=1)
        mean_betamap_pos = np.flipud(np.mean(img_data[:, :, :, won_bool], 
                                             axis=3)[:, :, z_slice].T)
        mean_betamap_neg = np.flipud(np.mean(img_data[:, :, :, ~won_bool], 
                                             axis=3)[:, :, z_slice].T)
        
        # Draw and store carpetplot
        fig = plt.figure(figsize=(15, 8), facecolor="w")

        gs = fig.add_gridspec(3, 4, height_ratios=[1, 3, 3])
        ax0 = fig.add_subplot(gs[0, :])
        ax1 = fig.add_subplot(gs[1, :])
        ax21 = fig.add_subplot(gs[2, 0])
        ax22 = fig.add_subplot(gs[2, 1])
        ax23 = fig.add_subplot(gs[2, 2])
        ax24 = fig.add_subplot(gs[2, 3])

        # FD timecourse
        ax0.plot(df_conf["framewise_displacement"], label="FD", color="#999999")
        ax0.set_xlim(min(df_conf.index), max(df_conf.index))
        ax0.set_xlabel("seconds")
        ax0.set_title(f"Subject: {sub} / Condition: {con_name}", fontSize=15)

        divider = make_axes_locatable(ax0)
        ax = divider.append_axes("right", size="5%", pad=0.05)
        ax.axis("off")
        ax0.legend()
        
        # Carpet
        im1 = ax1.imshow(
            carpet, 
            aspect="auto", 
            clim=clim_carpet, 
            cmap="RdBu_r", 
            interpolation="none"
        )
        ax1.set_ylabel("Voxel")
        ax1.set_xlabel("Trial")
        aligned_imshow_cbar(ax1, im1)

        # Mean beta maps
        im21 = ax21.imshow(
            mean_betamap_pos, 
            cmap="RdBu_r", 
            clim=clim_mean_betamaps,
            interpolation="gaussian"
        )
        im22 = ax22.imshow(
            mean_betamap_neg, 
            cmap="RdBu_r", 
            clim=clim_mean_betamaps,
            interpolation="gaussian",
        )
        ax21.set_title("+PE trials")
        ax22.set_title("-PE trials")
        ax21.annotate(f"z = {z_mni}", (3, 5))
        ax22.annotate(f"z = {z_mni}", (3, 5))
        aligned_imshow_cbar(ax21, im21)
        aligned_imshow_cbar(ax22, im22)

        # Beta value histograms
        ax23.hist(b_voxels, color="#bdbdbd", bins=100, range=hist_range)
        ax24.hist(b_trials, color="#bdbdbd")
        ax23.set_title("Mean voxels")
        ax24.set_title("Mean trials")
        ax23.set_xlabel(r"$\beta$ values")
        ax24.set_xlabel(r"$\beta$ values")
        
        plt.tight_layout()

        fig_fname = f"sub-{sub}_task-{con_name}_betamap_carpetplot.png"
        fig.savefig(join(path_betamaps_figures, fig_fname))
        plt.close(fig)